In [ ]:
# Setup
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
from datetime import datetime

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
      if hasattr(layer, 'reset_parameters'):
          print(f'Reset trainable parameters of layer = {layer}')
          layer.reset_parameters()

def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device
    
class Net_d(nn.Module):
    """
    3-layer CNN network with max pooling
    
    Args:
        in_channels: number of features of the input image ("depth of image")
        hidden_channels: number of hidden features ("depth of convolved images")
        out_features: number of features in output layer
    """
    
    def __init__(self, in_features, hidden_dim, out_features):
        super(Net_d, self).__init__()

        self.layer1 = nn.Linear(in_features, hidden_dim[0])
        self.relu1 = nn.ReLU()
        
        self.layer2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.relu2 = nn.ReLU()
        
        self.layer3 = nn.Linear(hidden_dim[1], hidden_dim[2])
        self.relu3 = nn.ReLU()
        
        self.layer4 = nn.Linear(hidden_dim[2], hidden_dim[3])
        self.relu4 = nn.ReLU()
        
        self.fc = nn.Linear(hidden_dim[3], out_features)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.layer3(x)
        x = self.relu3(x)
        x = self.layer4(x)
        x = self.relu4(x)
        x = self.fc(x)
        
        return x   
        
def custom_loss_d(y_pred, y_batch):
    
    loss = torch.mean(torch.abs(y_batch - y_pred))*128*2
    
    constraint = torch.sum(torch.abs(-4 - torch.sum(y_pred, axis = 1)))/(y_batch.shape[0])
    
    return (loss + constraint)

def custom_loss_d1(y_pred, y_batch):
    
    loss = torch.mean(torch.abs(y_batch - y_pred))
    
    constraint = (1/128)*torch.sum(torch.abs(-4 - torch.sum(y_pred, axis = 1)))/(y_batch.shape[0])
    
    return (loss + constraint)

    
def custom_loss_2(y_pred, y_batch):
    
    loss = torch.mean(torch.abs(y_batch - y_pred))*128
    
    constraint = torch.sum(torch.abs(-4 - torch.sum(y_pred, axis = 1)))/(y_batch.shape[0])/64
    
    return (loss + constraint)
    

def custom_loss_3(y_pred, y_batch):
    
    loss = torch.square(torch.mean(torch.abs(y_batch - y_pred))*128)
    
    constraint = torch.square(torch.sum(torch.abs(-4 - torch.sum(y_pred, axis = 1)))/(y_batch.shape[0]))
    
    return (loss + constraint)
    
def custom_loss_4(y_pred, y_batch):
    
    loss = torch.mean(torch.abs(y_batch - y_pred))*256
    
    constraint = torch.sum(torch.abs(-4 - torch.sum(y_pred, axis = 1)))/(y_batch.shape[0])
    
    return (loss + constraint)

def evaluate_accuracy(data_loader, net, device):
    """Evaluate accuracy of a model on the given data set."""
    net.to(device)
    net.eval()  #make sure network is in evaluation mode

    #init
    loss_sum = torch.tensor([0], dtype=torch.float32, device=device)
    con_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
         
            loss_sum += torch.sum(torch.abs(net(X) - y))/128
            con_sum += torch.sum(torch.abs(torch.sum(net(X), axis = 1) - torch.sum(y,axis = 1)))/128
            
            n += y.shape[0] #increases with the number of samples in the batch
    return loss_sum.item()/n, con_sum.item()/n

def train_test(epochs, net, device, train_loader, test_loader, optimizer, criterion):
    """Training and testing function of FCN"""
    
    startTime = datetime.now()
    
    # Define list to store losses and performances of each iteration
    train_losses = []
    train_losss = []
    test_losss = []
    train_cons = []
    test_cons = []
    
    for epoch in range(epochs):

        # Network in training mode and to device
        net.train()
        net.to(device)
        
        # Training loop
        for i, (x_batch, y_batch) in enumerate(train_loader):

            # Set to same device
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Set the gradients to zero
            optimizer.zero_grad()

            # Perform forward pass
            y_pred = net(x_batch)

            # Compute the loss
            loss = criterion(y_pred, y_batch)
            train_losses.append(loss.detach().cpu().numpy())
            
            # Backward computation and update
            loss.backward()
            optimizer.step()

        print(datetime.now() - startTime)
        
        if epoch == 25: 
            learning_rate = 0.0001
            optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.0)
        elif epoch == 50:
            learning_rate = 0.00001
            optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.0)
        elif epoch == 75:
            learning_rate = 0.00001
            optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.0) 
            
        if  epoch == 0 or epoch%5 == 0:
            # Compute train and test error
            train_loss, train_con = evaluate_accuracy(train_loader, net, device)
            test_loss, test_con = evaluate_accuracy(test_loader, net, device)
            
            # Development of performance
            train_losss.append(train_loss)
            test_losss.append(test_loss)
            
            train_cons.append(train_con)
            test_cons.append(test_con)
        
            # Print performance
            print('Epoch: {:.0f}'.format(epoch+1))
            print('Loss on train set: {:.7f}'.format(train_loss))
            print('Loss on test set: {:.7f}'.format(test_loss))
            print('Con on train set: {:.7f}'.format(train_con))
            print('Con on test set: {:.7f}'.format(test_con))
            print('')

    print(datetime.now() - startTime)
    
    return net, train_losses, train_losss, test_losss, train_cons, test_cons


In [ ]:
in_features = 222
in_features_qxyz = 296
in_features_H2O = 672
out_features = 128

# default
hidden_dim_d = [1024, 1024, 1024, 1024]

net_d = Net_d(in_features, hidden_dim_d, out_features)

# neurons
hidden_dim_2 = [1024, 512, 256, 128]
hidden_dim_3 = [512, 512, 512, 512]
hidden_dim_4 = [1524, 1524, 1524, 1524]

net_2 = Net_d(in_features, hidden_dim_2, out_features)
net_3 = Net_d(in_features, hidden_dim_3, out_features)
net_4 = Net_d(in_features, hidden_dim_4, out_features)

# layers
hidden_dim_5 = [1024, 1024, 1024]
hidden_dim_6 = [1024, 1024, 1024, 1024, 1024]

net_5 = Net_2(in_features, hidden_dim_5, out_features)
net_6 = Net_3(in_features, hidden_dim_6, out_features)

# activation function
net_7 = Net_act_1(in_features, hidden_dim_d, out_features)

# qxyz
net_9 = Net_d(in_features_qxyz, hidden_dim_d, out_features)
# H2O
net_10 = Net_d(in_features_H2O, hidden_dim_d, out_features)

epochs = 101

# optimizer
learning_rate_d = 0.001
optimizer_d = torch.optim.Adam(net_d.parameters(), lr=learning_rate_d, weight_decay=0.5)

hidden_dim_11 = [750, 750, 750, 1024, 1024]
net_11 = Net_10(in_features, hidden_dim_11, out_features)

# loss function
criterion_d = custom_loss_d
criterion_d1 = custom_loss_d1
criterion_2 = custom_loss_2
criterion_3 = custom_loss_3
criterion_4 = custom_loss_4

device = try_gpu()

In [ ]:
train_loader_H2O = torch.load('train_loader_XH2O_3D')
test_loader_H2O = torch.load('test_loader_XH2O_3D')

net_10.apply(reset_weights)
optimizer_d = torch.optim.Adam(net_10.parameters(), lr=learning_rate_d, weight_decay=0.5)
net_H2O, train_losses_H2O, train_losss_H2O, test_losss_H2O, train_cons_H2O, test_cons_H2O = train_test(epochs, net_10, device, train_loader_H2O, test_loader_H2O, optimizer_d, criterion_d)
torch.save(net_H2O,'net_H2O_2')
np.savez('net_H2O_2.npz', trainlossH2O=train_losses_H2O, trainlosssH2O=train_losss_H2O, testlosssH2O=test_losss_H2O, trainconH2O=train_cons_H2O, testconH2O=test_cons_H2O)


Reset trainable parameters of layer = Linear(in_features=672, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=128, bias=True)
0:03:45.616269
Epoch: 1
Loss on train set: 0.0338394
Loss on test set: 0.0399452
Con on train set: 0.0010451
Con on test set: 0.0007874

0:07:27.281884
0:11:03.461363
0:14:29.691009
0:17:48.049084
0:21:07.882130
Epoch: 6
Loss on train set: 0.0321993
Loss on test set: 0.0402334
Con on train set: 0.0010559
Con on test set: 0.0015022

0:24:25.457133
0:27:41.166943
0:30:57.017458
0:34:16.362605
0:37:33.679601
Epoch: 11
Loss on train set: 0.0318426
Loss on test set: 0.0397516
Con on train set: 0.0011162
Con on test set: 0.0009673

0:40: